In [1]:
from pathlib import Path

In [ ]:
Path.cwd()
dir1= Path('folder_test1')
dir2= Path('folder_test2')

In [4]:
dir1.mkdir(exist_ok=True)
dir2.mkdir()

In [15]:
file1=dir1/'file1.txt'

SyntaxError: unexpected character after line continuation character (3181518727.py, line 1)

In [6]:
file1.touch()

In [7]:
file1.exists()

True

In [13]:
file1.rename(dir2/file1.name)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'folder_test1\\file1.txt' -> 'folder_test2\\file1.txt'

In [14]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import util
from training_functions import create_model,train_model
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, VerticalFlip
import os
from os import path
import random as r
import imageio
from import_augmentation_function import import_fun, aug_fun, import_fun_neg, normalization_fun_glob, normalization_fun_loc, normalization_fun_g, import_aug_fun
import tensorflow as tf 

data_ratio= 0.1
data_split_state = None


def load_aug_train(files_dir, images_dir,images_neg_dir, sigma, number_of_augmentations):
    date, dye, cell_type, microscope, bf_fl, pos_neg = images_dir.split('_')
    joined_path = os.path.join(files_dir, images_dir)
    joined_path_neg = os.path.join(files_dir, images_neg_dir)

    all_image_array, all_image_array_gauss= import_fun(joined_path, files_dir, images_dir,sigma)
    all_image_array_gauss, all_image_array= zero_frames(all_image_array.shape[1], all_image_array_gauss, all_image_array)
    all_image_array_neg, all_image_array_gauss_neg = import_fun_neg(joined_path_neg,files_dir, images_neg_dir)

    all_image_array = np.concatenate((all_image_array,all_image_array_neg))
    all_image_array_gauss = np.concatenate((all_image_array_gauss,all_image_array_gauss_neg))

                            ## NORMALIZATION ##

    #norm_image_array= normalization_fun_glob(all_image_array)
    norm_image_array= normalization_fun_loc(all_image_array, 21, 5)
    norm_image_array_gauss = normalization_fun_g(all_image_array_gauss, 0.1)

                            ## AUGMENTATION ##

    augmentation_data, validation_data, augmentation_data_gauss, validation_data_gauss =  train_test_split(norm_image_array, norm_image_array_gauss,
                                                                                                       test_size=data_ratio, random_state=data_split_state)
    data_aug = np.zeros((np.size(augmentation_data , 0),128,128))
    data_gauss_aug = np.zeros((np.size(augmentation_data , 0),128,128))
    data_val = np.zeros((np.size(validation_data , 0),128,128))
    data_gauss_val = np.zeros((np.size(validation_data_gauss , 0),128,128))

    for frame_index in range(np.size(data_aug , 0)):
        data_aug[frame_index,:,:] = augmentation_data[frame_index, 64:192 , 64:192]
        data_gauss_aug[frame_index,:,:] = augmentation_data_gauss[frame_index, 64:192 , 64:192]
    for frame_index in range(np.size(data_val , 0)):
        data_val[frame_index,:,:] = validation_data[frame_index, 64:192 , 64:192]
        data_gauss_val[frame_index,:,:] = validation_data_gauss[frame_index, 64:192 , 64:192]
    
    for j in range(number_of_augmentations):
        transform = Compose([Rotate(limit=45, p=0.5), RandomRotate90(p=0.5), HorizontalFlip(p=0.5), Flip(p=0.5), VerticalFlip(p=0.5)])
        print('augmentation', j)
        augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss,transform)
        data_aug = np.concatenate((data_aug, augment_data[:, 64:192 , 64:192]))
        data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss[:, 64:192 , 64:192]))

                                ## SHUFFLE ##

    shuffle_array_val= np.arange(0, data_val.shape[0], 1)
    shuffle_array_aug= np.arange(0, data_aug.shape[0], 1)
    r.shuffle(shuffle_array_val)
    r.shuffle(shuffle_array_aug)

    data_val=data_val[shuffle_array_val]
    data_gauss_val=data_gauss_val[shuffle_array_val]
    data_aug=data_aug[shuffle_array_aug]
    data_gauss_aug=data_gauss_aug[shuffle_array_aug]

'folder_test1\\file1.txt'

In [28]:
import tifffile
from PIL import Image
import numpy as np
img = Image.open(r'C:\Users\roumba\Documents\Software\deep-events\original_data\220921_mtstaygold_cos7_iSIM_fl\220921_mtstaygold_cos7_Fluorescent_4.ome.tif')
dataar=np.zeros((100, 1268,1268))

for frame_index in range(100):
    img.seek(frame_index) #starts from 0 I think?
    yl=2048-2000
    yu=1268+yl
    box_a = (512,yl,1780,yu) #choose dimensions of box
    imcrop= img.crop(box_a)
    dataar[frame_index, :, :] = np.array(imcrop)
        

tifffile.imwrite('original_crop.tiff' , (dataar).astype(np.uint16), photometric='minisblack')


In [49]:
img = Image.open(r'C:\Users\roumba\Documents\Software\deep-events\original_data\220921_mtstaygold_cos7_iSIM_fl\220921_mtstaygold_cos7_Fluorescent_4.ome.tif')
data_norm=np.zeros((img.n_frames,2048,2048))
k=0.3
kk=1/(1-k)
data_first=np.zeros((img.n_frames,2048,2048))

for framenumber in range(img.n_frames):
    img.seek(framenumber)
    arrayim=np.array(img)
    data_first[framenumber,:,:]=arrayim
    data = (data_first[framenumber])/255                                            
    data = data-k
    data [data < 0] = 0   
    data_norm[framenumber] = data*kk

import imageio
imageio.mimwrite('original_norm.tiff', (data_norm).astype(np.float64))


In [48]:
from skimage.filters import threshold_otsu, threshold_local,rank
from skimage.morphology import disk
from skimage.util import img_as_ubyte
import cv2 as cv

final_loc=np.zeros((np.size(data_aug,0),128,128))
final_loc_bin=np.zeros((np.size(data_aug,0),128,128))
final_glob=np.zeros((np.size(data_aug,0),128,128))
final_cv=np.zeros((np.size(data_aug,0),128,128))

radius = 31
block_size = 51
footprint = disk(radius)

for framenumber in range(np.size(data_aug, 0)):
    image=data_aug[framenumber]
    image_loc=data_aug[framenumber]
    image_loc_bin=data_aug[framenumber]
    img = (image_loc)/(np.max(image_loc))
    img = img_as_ubyte(img)

    threshold= threshold_otsu(image)
    image[image < threshold] = 0
    # binary_global = image > threshold
    final_glob[framenumber,:,:]= image

    local_otsu = rank.otsu(img, footprint)
    image_loc[image_loc < local_otsu] = 0
    # local_image=img >= local_otsu
    final_loc[framenumber,:,:]=image_loc

    local_thresh = threshold_local(image_loc_bin, block_size, offset=5)
    # binary_local = image > local_thresh
    image_loc_bin[image_loc_bin < local_thresh] = 0
    final_loc_bin[framenumber,:,:]=image_loc_bin

    # thresh4 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    # final_cv[framenumber,:,:]=thresh4

    

import napari
viewer=napari.Viewer()
viewer.add_image(final_loc)
viewer.add_image(final_loc_bin)
viewer.add_image(final_glob)
# viewer.add_image(final_cv)
viewer.show()



[[[0.30882353 0.28921569 0.32843137 ... 0.32352941 0.28921569 0.26960784]
  [0.35294118 0.24509804 0.30882353 ... 0.30392157 0.26470588 0.28431373]
  [0.34803922 0.29411765 0.31862745 ... 0.28921569 0.29901961 0.26960784]
  ...
  [0.26960784 0.25       0.24509804 ... 0.29411765 0.30392157 0.29901961]
  [0.25490196 0.25490196 0.28431373 ... 0.28921569 0.25980392 0.25980392]
  [0.31862745 0.24509804 0.28921569 ... 0.26470588 0.25490196 0.28431373]]

 [[0.28921569 0.30392157 0.35294118 ... 0.28921569 0.25980392 0.28921569]
  [0.28431373 0.37254902 0.32352941 ... 0.29901961 0.28921569 0.25      ]
  [0.33333333 0.31372549 0.33333333 ... 0.26470588 0.28921569 0.24019608]
  ...
  [0.26470588 0.25490196 0.30882353 ... 0.26470588 0.26960784 0.29411765]
  [0.26960784 0.2745098  0.30392157 ... 0.29411765 0.26470588 0.26960784]
  [0.29411765 0.27941176 0.24509804 ... 0.25980392 0.26470588 0.24019608]]

 [[0.28921569 0.33823529 0.29411765 ... 0.24019608 0.29901961 0.28431373]
  [0.28921569 0.357843